In [0]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

# Text Generation using a Recurrent Neural Network

- We will work with a dataset of Shakespeare's writing
- Build a model with `tf.keras` to analyze the sequence of characters
- Apply the model to write new text in Shakespeare's style
- Use the **eager execution** mode of TensorFlow to achieve immediate computation

This project is adapted from [TensorFlow tutorial](https://www.tensorflow.org/tutorials/sequences/text_generation)

In [0]:
# Get the text file from:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

## Read the data

In [42]:
# Read the text as a string
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [43]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [44]:
# The unique characters in the file
# Python set is a data structure containing unique elements
vocab = sorted(set(text))
# Print with formatted string: {index:format}
print ('{} unique characters'.format(len(vocab)))

65 unique characters


## Vectorize the text

In [0]:
# Creating a mapping from unique characters to indices
# enumerate: returns index and the value
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [46]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [47]:
# Show how the first 13 characters from the text are mapped to integers
# repr: string representation of an object
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
# Dataset.from_tensor_slices(): convert a numpy array to tf Dataset
# Dataset.take(): create a sub Dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

Instructions for updating:
Colocations handled automatically by placer.
F
i
r
s
t


In [10]:
# batch(): cut the dataset into chunks
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# join(): concatenate a list of elements and form a string
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


## Match input character with output character

In [0]:
# Match input with output
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# map(): similar to pandas.DataFrame.apply()
dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


## Shuffle and create training batches

In [14]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build the training model

- Embedding layer
- GRU layer

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## Try the pre-trained model

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([33, 33, 36, 16, 29,  0, 19, 47, 13, 51, 54, 48, 28, 59,  7, 23, 47,
       20, 35, 23, 13, 54, 44, 43, 63, 46, 61, 18, 51, 17, 54, 40, 54, 30,
        9, 26, 60, 53, 13,  7, 28, 11, 29,  4, 38,  5, 17, 32,  7, 21, 45,
        2,  1, 45, 63, 12, 51,  4, 53,  1, 55, 47, 52, 57, 15, 44, 17, 29,
       54, 45, 30, 27, 59, 37,  8, 33, 34, 61, 56, 21, 46, 42, 45, 61, 56,
       10, 51, 62, 49, 49, 25, 60, 54, 28, 10, 62, 34, 39, 54, 53])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ssion of thy sounds,\nThou madst thine enemies shake, as if the world\nWere feverous and did tremble.\n'

Next Char Predictions: 
 "UUXDQ\nGiAmpjPu-KiHWKApfeyhwFmEpbpR3NvoA-P;Q&Z'ET-Ig! gy?m&o qinsCfEQpgROuY.UVwrIhdgwr:mxkkMvpP:xVapo"


## Train the model

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174727


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=3

In [28]:
history = model.fit(dataset.repeat(),
                    epochs=EPOCHS,
                    steps_per_epoch=steps_per_epoch,
                    callbacks=[checkpoint_callback])
# callbacks: a list of functions to be applied after each epoch

Epoch 1/3
173/174 [============================>.] - ETA: 0s - loss: 2.7149WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
174/174 [==============================] - 13s 74ms/step - loss: 2.7115
Epoch 2/3
174/174 [==============================] - 11s 65ms/step - loss: 1.9541
Epoch 3/3
174/174 [==============================] - 11s 66ms/step - loss: 1.6879


## Generate new text

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# build(): build the model on given input size
model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  # expand_dims(): add an trivial dimension
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      # squeeze(): remove all size 1 dimensions
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"ROMEO: "))

Instructions for updating:
Use tf.random.categorical instead.
ROMEO: is thy faid.

ROMPOY:
my light done fate than she sometione.

COMINIUS:

'TERSIO:
Go, siren.

CORIOLANUS:
Not it in MARTARUS:
Theng, fon that I dear, thy dauchs
condentle I will bad effremy his told eath me.

GRUMIO:
But of the pisits of you be us alfown? Shall pride master yourmate no trus at
Her fear at:
Nut discoons, they for a benderoun besure;
And well they Bus Grem of sole cut destreicalatious wilt so enceant wifter.

GLOUCESTER:
Brother, hew dis in the wint' therefore it.

Yerder:
By hisure Princorvener.

Plought.

hearted the grave a vellefure:
Yet you mest you, E play their wisom and queen.
Where my fastilless.

MARCIUT:
ay, sir, the callr here if now,
God thinks, or these quarner his canned
this us anow lear't be busine ot o'er breath.
Than I pury, makes.
Came, Marin be hing, sect there Ipanied:
And will make that my tear bore, sir, my more.

DIOND:
Bolders of my Fartue, and there neight on them.

Pillt:
She

## Analysis:

- Since the model is character-based, the model did not know how to spell an English word.
- The structure of the output resembles a play: blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- The output displays a balanced mixture of short words and long words. Paragraphs always end with a period.